In [2]:
import pandas
from sklearn import preprocessing
# enc = preprocessing.LabelEncoder()
# res = enc.fit(train['Weekday'])
# print res.classes_
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer

def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(outtype='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df

train = pandas.read_csv('train.csv')

def process_train_data(train):
    train.drop('VisitNumber', 1)
    df = encode_onehot(train, cols=['Weekday', 'DepartmentDescription'])
    processed_df = DataFrame()

    # One Hot Encoding for features
    for k in df.keys():
        if k.startswith('DepartmentDescription='):
            processed_df[k] = df.groupby(['VisitNumber'])[k].sum()
        if k.startswith('Weekday='):
            processed_df[k] = df.groupby(['VisitNumber'])[k].max()

    # Process all the class label to categorical values
    train['TripTypeStr'] = train['TripType'].apply(str)
    processed_train = encode_onehot(train, cols=['TripTypeStr'])
    cols_rename = {}

    for k in processed_train.keys():
        if k.startswith('TripTypeStr='):
            cols_rename[k] = k.replace('Str=', '_')
    processed_train.rename(columns=cols_rename,inplace=True)

    for k in processed_train.keys():
        if k.startswith('TripType_'):
            processed_df[k] = processed_train.groupby('VisitNumber')[k].sum()
    return processed_df

processed_df = process_train_data(train)

/Users/ngoc/.venv/lib/python2.7/site-packages/ipykernel/__main__.py:28: FutureWarning: the 'outtype' keyword is deprecated, use 'orient' instead


In [3]:
print processed_df.keys()

Index([u'DepartmentDescription=1-HR PHOTO',
       u'DepartmentDescription=ACCESSORIES',
       u'DepartmentDescription=AUTOMOTIVE', u'DepartmentDescription=BAKERY',
       u'DepartmentDescription=BATH AND SHOWER',
       u'DepartmentDescription=BEAUTY', u'DepartmentDescription=BEDDING',
       u'DepartmentDescription=BOOKS AND MAGAZINES',
       u'DepartmentDescription=BOYS WEAR',
       u'DepartmentDescription=BRAS & SHAPEWEAR',
       ...
       u'TripType_41', u'TripType_42', u'TripType_43', u'TripType_44',
       u'TripType_5', u'TripType_6', u'TripType_7', u'TripType_8',
       u'TripType_9', u'TripType_999'],
      dtype='object', length=113)


In [4]:
test = pandas.read_csv('test.csv')

def process_test_data(test_data):
    df = encode_onehot(train, cols=['Weekday', 'DepartmentDescription'])
    processed_df = DataFrame()

    # One Hot Encoding for features
    for k in df.keys():
        if k.startswith('DepartmentDescription='):
            processed_df[k] = df.groupby(['VisitNumber'])[k].sum()
        if k.startswith('Weekday='):
            processed_df[k] = df.groupby(['VisitNumber'])[k].max()

    return processed_df
process_test_df = process_test_data(test)

/Users/ngoc/.venv/lib/python2.7/site-packages/ipykernel/__main__.py:28: FutureWarning: the 'outtype' keyword is deprecated, use 'orient' instead


In [4]:
print process_test_df.keys()

Index([u'DepartmentDescription=1-HR PHOTO',
       u'DepartmentDescription=ACCESSORIES',
       u'DepartmentDescription=AUTOMOTIVE', u'DepartmentDescription=BAKERY',
       u'DepartmentDescription=BATH AND SHOWER',
       u'DepartmentDescription=BEAUTY', u'DepartmentDescription=BEDDING',
       u'DepartmentDescription=BOOKS AND MAGAZINES',
       u'DepartmentDescription=BOYS WEAR',
       u'DepartmentDescription=BRAS & SHAPEWEAR',
       u'DepartmentDescription=CAMERAS AND SUPPLIES',
       u'DepartmentDescription=CANDY, TOBACCO, COOKIES',
       u'DepartmentDescription=CELEBRATION',
       u'DepartmentDescription=COMM BREAD',
       u'DepartmentDescription=CONCEPT STORES',
       u'DepartmentDescription=COOK AND DINE', u'DepartmentDescription=DAIRY',
       u'DepartmentDescription=DSD GROCERY',
       u'DepartmentDescription=ELECTRONICS',
       u'DepartmentDescription=FABRICS AND CRAFTS',
       u'DepartmentDescription=FINANCIAL SERVICES',
       u'DepartmentDescription=FROZEN FOODS'

In [5]:
targets = [k for k in processed_df.keys() if k.startswith('TripType_')]
features = [k for k in processed_df.keys() if not k.startswith('TripType_') ]

In [ ]:
train_X = processed_df[features].as_matrix()
print len(train_X['TripType_41'])
# import xgboost
# for k in processed_df.keys():
#     train_y = processed_df['TripType_41'].as_matrix()
#     gbm = xgboost.XGBClassifier(
#         max_depth=3,
#         n_estimators=300,
#         learning_rate=0.05
#     ).fit(train_X, train_y)

In [10]:
test_X = process_test_df[features].as_matrix()
# predictions = gbm.predict(test_X)
# print predictions

ValueError: bad input shape (95674, 38)